<a href="https://colab.research.google.com/github/WilliamAshbee/splineexample/blob/main/LSTM_mcfixed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

With progressive growing of the sequence, the loss drops more stably. 

https://r2rt.com/non-zero-initial-states-for-recurrent-neural-networks.html

In [1]:
import torch
import numpy as np
import pylab as plt
from skimage import filters
import math

global numpoints
numpoints = 1000
side = 32

rows = torch.zeros(32,32)
columns = torch.zeros(32,32)


for i in range(32):
    columns[:,i] = i
    rows[i,:] = i


def donut_matrix(length = 10):
    radiusMax = side /3
    w = 1
    sigmas = [None, 1]
    
    canvas = torch.zeros((length,side, side))
    r0 = torch.tensor(np.random.uniform(side/4, side/3, length))

    radii = torch.zeros((length,numpoints))
    radii[:, :] = r0.unsqueeze(1)
    
    ind = [x for x in range(numpoints)]

    theta = torch.FloatTensor(ind)
    theta *= math.pi*2.0/(float)(numpoints)
    
    for i in range(1,length):
        a = np.random.uniform(1.0,3.0)*torch.sin(np.random.uniform(20.0)*theta+np.random.uniform(1000.0))
        #a = 4.0*torch.sin(10.0*theta)
        #print(a.shape,torch.max(a))
        radii[i,:] += a
        #print(radii.shape, torch.max(radii))
    
    assert torch.min(radii)>0
    #print(radii.max(axis = 0)[0].shape)
    rmaxs = radii.max(axis = 1)[0]
    pmins = rmaxs+1.0
    pmaxs = side-rmaxs-1.0
    x0 = np.random.uniform(pmins,pmaxs)
    y0 = np.random.uniform(pmins,pmaxs)
    x0[:]=side/2
    y0[:]=side/2
    x0 = torch.tensor(x0)
    y0 = torch.tensor(y0)
    
    x0 = x0.unsqueeze(1)
    y0 = y0.unsqueeze(1)
    #radii = torch.from_numpy(radii)
    xrfactors = torch.cos(theta).unsqueeze(0)
    yrfactors = torch.sin(theta).unsqueeze(0)
    
    print('x0_y0_r_xrf_yrf',x0.shape,y0.shape,radii.shape,xrfactors.shape,yrfactors.shape)

    x = (x0+(xrfactors*radii))
    y = (y0+(yrfactors*radii))
    assert x.shape == (length,numpoints)
    assert y.shape == (length,numpoints)
    assert torch.sum(x[x>(side-1)])==0 
    assert torch.sum(x[x<0])==0 
    assert torch.sum(y[y>(side-1)])==0 
    assert torch.sum(y[y<0])==0 
    
    points = torch.zeros(length,numpoints,2)
    for l in range(length):
        canvas[l,y[l,:].type(torch.LongTensor),x[l,:].type(torch.LongTensor)]=1.0
        points[l,:,0] = x[l,:]#modified for lstm discriminator
        points[l,:,1] = y[l,:]#modified for lstm discriminator 
    
    
    return {
        'canvas': canvas, 
        'points':points.type(torch.FloatTensor)}

def plot_all_model( sample = None, label=None):
    X = label[:,0]
    Y = label[:,1]
    img = sample[:,:].squeeze().cpu().numpy()
    plt.imshow(img, cmap=plt.cm.gray_r)
    predres = 1000

    s = [.001 for x in range(predres)]
    
    assert len(s) == predres
    c = ['red' for x in range(predres)]
    assert len(c) == predres
    plt.scatter(X.cpu().numpy(),Y.cpu().numpy(),s = s,c = c)


def getXYs(model=None,loader_disp = None):

    outputs = torch.zeros(100,1000,2).cuda()
    with torch.no_grad():
      h0 = None
      c0 = None
      X = []
      Y = []
      for samples,labels in loader_disp:
        for i in range(11):
          optimizer.zero_grad()
          if i == 0:
            out, h0, c0,o,output = model(samples.cuda(),labels[:,0,:2].cuda(),h0,c0,ind = i)
            outputs[:,i*50:(i+1)*50,:] = out.detach().clone()
          else:
            out, h0, c0,o,output = model(samples.cuda(),labels[:,0,:2].cuda(),h0,c0,ind = i,o = o,output=output)
            outputs[:,i*50:(i+1)*50,:] = out.detach().clone()

        return samples, outputs

          #X.extend(out.detach()[0,:,0].tolist())
          #Y.extend(out.detach()[0,:,1].tolist())

          
        

def plot_all( sample = None, labels = None):
    img = sample[:,:].squeeze().cpu().numpy()
    plt.imshow(img, cmap=plt.cm.gray_r)
    X = labels[:,0]
    Y = labels[:,1]
    s = [.001 for x in range(numpoints)]
    c = ['red' for x in range(numpoints)]
    plt.scatter(X.cpu().numpy(),Y.cpu().numpy(),s = s,c = c)
    
class DonutDataset(torch.utils.data.Dataset):
    """Donut dataset."""
    def __init__(self, length = 10):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.length = length
        self.values = donut_matrix(length)
        assert self.values['canvas'].shape[0] == self.length
        assert self.values['points'].shape[0] == self.length
        
        count = 0
        for i in range(self.length):
          a = self[i]
          c = a[0]
          for el in a[1]:
            #print(c[(int)(el[1]),(int)(el[0])].item())
            #assert c[(int)(el[1]),(int)(el[0])].item() == 1
            y,x = (int)(el[1]),(int)(el[0])
            if x < side-2 and x > 2 and y < side-2 and y > 2: 
              if c[y,x] != 1 and \
                c[y+1,x] != 1 and c[y+1,-1+x] != 1 and c[y+1,1+x] != 1 and \
                c[y-1,x] != 1 and c[y,-1+x] != 1 and c[y,1+x] != 1:
                count+=1
        assert count ==0
    def __len__(self):
        return self.length


    def __getitem__(self, idx):
        canvas = self.values["canvas"]
        
        canvas = canvas[idx,:,:]
        assert canvas.shape == (side,side)
        
        points = self.values["points"]
        points = points[idx,:]
        #points = points.unsqueeze(1)
        #z = torch.zeros(numpoints,1)
        #print(z.shape)
        #points = torch.cat([points,z], dim = 1)
        
        #print('points', points.shape)
        return canvas, points
    
    @staticmethod
    def displayCanvas(title,dataset, model):
        #model.setBatchSize(batch_size = 1)
        if model != None:
          model = model.eval()
          loader_disp = data.DataLoader(
            dataset, 
            batch_size=100,
            num_workers=2)
          samples, outputs = getXYs(model=model,loader_disp = loader_disp)
          for i in range(100):
            plt.subplot(10,10,i+1)
            plot_all_model(samples[i,:,:],outputs[i,:,:])
            plt.axis('off')

        else:
          for i in range(100):
            sample, labels = dataset[i]
            plt.subplot(10,10,i+1)
            plot_all(sample = sample,labels = labels)
            plt.axis('off')

        plt.savefig(title,dpi=450)
        plt.clf()
        plt.cla()
        plt.close()#should free memory
        print('open file: ',title)
dataset = DonutDataset(length = 100)

DonutDataset.displayCanvas('donut.png',dataset, model = None)


x0_y0_r_xrf_yrf torch.Size([100, 1]) torch.Size([100, 1]) torch.Size([100, 1000]) torch.Size([1, 1000]) torch.Size([1, 1000])
open file:  donut.png


In [2]:

from torch.utils import data
from torch.utils.data import DataLoader, TensorDataset, RandomSampler

mini_batch = 100
dataset = DonutDataset(length = 100*20)
loader_train = data.DataLoader(
    dataset, 
    batch_size=mini_batch,
    sampler=RandomSampler(data_source=dataset),
    num_workers=2)


x0_y0_r_xrf_yrf torch.Size([2000, 1]) torch.Size([2000, 1]) torch.Size([2000, 1000]) torch.Size([1, 1000]) torch.Size([1, 1000])


In [3]:
import torch
import torch.nn as nn
from torch.hub import load_state_dict_from_url

class DecoderRNN(nn.Module):
    def __init__(self):
        super(DecoderRNN, self).__init__()
        self.track = 0
        # define the properties
        self.embed_size = 2
        self.hidden_size = 512
        self.num_layers = 2
        self.seq_len = 50
        self.lm_len = 20
        self.full_len = 1000
        self.longtermMem = None
        self.longtermMemP = None
        self.lstm_cell = nn.LSTM(self.hidden_size, self.hidden_size,self.num_layers)

        self.inject_image =  nn.Sequential(
            nn.Linear(2+1024+3*self.hidden_size+1,self.hidden_size),
            nn.Dropout(),
            nn.Tanh()
        )

        
        self.fc_out = nn.Sequential(
            nn.Linear(2+self.hidden_size*4,2),
        #    nn.ReLU(),
        #    nn.Dropout(),
        #    nn.Linear(10000,2),
            nn.Sigmoid()
        )
        

        self.alpha_1 = nn.Sequential(
            nn.Linear(self.hidden_size,self.seq_len),
            nn.Softmax()
        )
        self.alpha_2 = nn.Sequential(
            nn.Linear(self.hidden_size,self.seq_len),
            nn.Softmax()
        )
        
        self.alpha_long1 = nn.Sequential(
            nn.Linear(self.hidden_size,self.lm_len),
            nn.Softmax()
        )
        self.alpha_long2 = nn.Sequential(
            nn.Linear(self.hidden_size,self.lm_len),
            nn.Softmax()
        )
        
        self.alpha_1 = nn.Sequential(
            nn.Linear(self.hidden_size,self.seq_len),
            nn.Softmax()
        )
        
        self.alpha_long1 = nn.Sequential(
            nn.Linear(self.hidden_size,self.lm_len),
            nn.Softmax()
        )

        self.embs = None #= torch.empty((self.batch_size, 1000, self.hidden_size)).cuda()
        
        
        

            
        
    def forward(self, features,p0,h0 = None, c0= None, ind = 0, o = None,output = None):
        assert features != None
        #assert captions != None
        #print('featursescaptions2',features.shape,captions.shape)
        #assert features.shape[0]==captions.shape[0]
        assert len(features.shape) == 3
        self.track +=1
        batch_size = features.shape[0]
        

        #img_emb = torch.empty((batch_size, captions.size(1), self.hidden_size)).cuda()
        
        p0 = torch.flatten(p0[:,:2],start_dim=1)
        

        if o == None:
          out = p0
          self.longtermMem = torch.zeros(batch_size,self.lm_len,self.hidden_size).cuda()
          self.embs = torch.empty((batch_size, self.seq_len, self.hidden_size)).cuda()
        else:
          out = o

        
        if h0 == None and c0 == None:
          h0 = torch.zeros(self.num_layers,batch_size,self.hidden_size).cuda()
          c0 = torch.zeros(self.num_layers,batch_size,self.hidden_size).cuda()

        hidden_state = h0
        cell_state = c0
        outputs = torch.empty((batch_size, self.seq_len, 2)).cuda()
        
        output = None

        for t in range(self.seq_len):
        
            # for the first time step the input is the feature vector
            if output == None:
              output = torch.zeros(batch_size,self.hidden_size).cuda()
              outesread1 = torch.zeros(batch_size,self.hidden_size).cuda()
              outesreadl1 = torch.zeros(batch_size,self.hidden_size).cuda()
            
            #print('1',torch.flatten(features,start_dim=1).shape,torch.flatten(self.longtermMemP,start_dim=1).shape,output.squeeze().shape,)
            
            t_ind = torch.zeros(batch_size,1).cuda()
            t_ind[:,:] = ind*50+t
            combin_lstm = torch.cat([torch.flatten(features,start_dim=1),out,output.view(batch_size,self.hidden_size),outesread1,outesreadl1,t_ind],dim=1)
            combin_lstm = self.inject_image(combin_lstm)
            
            a1 = self.alpha_1(combin_lstm)
            #print('a1',a1.shape,a1.unsqueeze(2).shape,self.embs.shape,outesread1.shape)
            outesreada1 = self.embs.clone()*a1.unsqueeze(2)#clone
            outesread1 = torch.sum(outesreada1,dim=1)
            
            #print('122',self.longtermMem.shape,outesread1.shape)
            #print('ind',ind)
            self.longtermMem[:, ind, :] = (outesread1).detach()
            
            al1 = self.alpha_long1(combin_lstm)
            outesreadal1 = self.longtermMem.clone()*al1.unsqueeze(2)#clone
            outesreadl1 = torch.sum(outesreadal1,dim=1)
            
            output, (hidden_state, cell_state) = self.lstm_cell(combin_lstm.unsqueeze(0), (hidden_state, cell_state))

            outputs[:, t, :] = out
            self.longtermMemP[:,50*ind+t,:] = out.detach()
            
            self.embs[:, t, :] = output.view(batch_size,self.hidden_size).clone().detach()
            

            
            a1 = self.alpha_1(combin_lstm)
            a2 = self.alpha_2(combin_lstm)
            a1 = (a1+a2)/2.0
            outesreada1 = self.embs.clone()*a1.unsqueeze(2)#clone
            outesread1 = torch.sum(outesreada1,dim=1)
            
            self.longtermMem[:, ind, :] = (outesread1).detach()
            
            al1 = self.alpha_long1(combin_lstm)
            al2 = self.alpha_long2(combin_lstm)
            al1 = (al1+al2)/2.0
            outesreadal1 = self.longtermMem.clone()*al1.unsqueeze(2)#clone
            outesreadl1 = torch.sum(outesreadal1,dim=1)
            
            

            
            combOut = torch.cat([combin_lstm, output.view(batch_size,self.hidden_size),out,outesread1,outesreadl1],dim=1)#,outesread],dim=1)
            

            out = self.fc_out(combOut)
            out = out*32.0
            # build the output tensor
        
        #self.embs = self.embs.detach().clone()
        return outputs, hidden_state.detach(), cell_state.detach(), out.detach(),output.detach()#, captions


In [4]:
#torch.stack([torch.zeros(64,10),torch.ones(64,10)],dim = 0)[0,:,:]

In [5]:
a = torch.ones(64,256,50)
b = torch.from_numpy(np.random.randn(50))
c = a*b
print(c[0,1,:])
print(c[1,1,:])

tensor([ 1.1634, -0.6587, -0.9364, -0.6733, -0.2407,  0.1692, -0.5019, -0.6993,
        -1.3511,  1.4785,  0.8538,  0.0778, -0.1455,  0.0605,  0.1074,  0.6239,
        -1.2108,  0.8102, -0.3429,  1.8962,  2.4320,  0.5717,  0.7469, -1.1871,
        -0.2033, -0.1689,  0.8417, -1.4504,  0.2744, -0.5405, -1.4982, -0.2523,
        -1.9696, -0.9513,  1.3946,  0.8669,  1.6528, -1.0605, -0.0638,  0.0394,
        -1.0104,  0.3164, -0.0487,  0.2336,  0.1623,  1.0802, -0.1026,  0.2080,
         0.3997, -1.6233], dtype=torch.float64)
tensor([ 1.1634, -0.6587, -0.9364, -0.6733, -0.2407,  0.1692, -0.5019, -0.6993,
        -1.3511,  1.4785,  0.8538,  0.0778, -0.1455,  0.0605,  0.1074,  0.6239,
        -1.2108,  0.8102, -0.3429,  1.8962,  2.4320,  0.5717,  0.7469, -1.1871,
        -0.2033, -0.1689,  0.8417, -1.4504,  0.2744, -0.5405, -1.4982, -0.2523,
        -1.9696, -0.9513,  1.3946,  0.8669,  1.6528, -1.0605, -0.0638,  0.0394,
        -1.0104,  0.3164, -0.0487,  0.2336,  0.1623,  1.0802, -0.1026,  

In [6]:
model = DecoderRNN().cuda()

In [7]:
optimizer = torch.optim.Adam(model.parameters(),lr = 0.0001, betas = (.9,.999))#ideal


In [8]:
import torch
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()


In [9]:
model = model.train()
for seql in range(3,12,4):
  #seql = 19
  for epoch in range (5):
    loss = None
    for x,y in loader_train:
      h0 = None
      c0 = None
      for i in range(seql):
        x = x.cuda()
        y = y.cuda()
        
        optimizer.zero_grad()
        
        if i == 0:
          out, h0, c0,o,output = model(x,y[:,0,:2],h0,c0,ind = i)
        else:
          out, h0, c0,o,output = model(x,y[:,0,:2],h0,c0,ind = i,o = o,output=output)
        loss = torch.mean((out-y[:,1+(50*i):1+(50*(i+1)),:2])**2)
        #writer.add_scalar("Loss/train", loss, epoch)
        
        loss.backward()
        optimizer.step()
      #print(y[:,0,:2])
      print('epoch',epoch,'seql',seql)
      print(loss)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/container.py:119: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


epoch 0 seql 3
tensor(31.2984, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 3
tensor(14.2929, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 3
tensor(5.9697, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 3
tensor(4.3709, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 3
tensor(6.4631, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 3
tensor(6.6490, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 3
tensor(5.1246, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 3
tensor(3.8932, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 3
tensor(3.0905, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 3
tensor(2.7928, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 3
tensor(2.4549, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 3
tensor(2.4079, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 3
tensor(2.4548, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 3
tensor(2.1953, device='cuda:0', grad_fn=<MeanBa

In [10]:
#optimizer = torch.optim.Adam(model.parameters(),lr = 0.0001, betas = (.9,.999))#ideal

for g in optimizer.param_groups:
    g['lr'] = 0.0001


In [11]:

model = model.train()
#for seql in range(15,20,4):
seql = 11
for epoch in range (15):
  loss = None
  for x,y in loader_train:
    h0 = None
    c0 = None
    for i in range(seql):
      x = x.cuda()
      y = y.cuda()
      
      optimizer.zero_grad()
      
      if i == 0:
        out, h0, c0,o,output = model(x,y[:,0,:2],h0,c0,ind = i)
      else:
        out, h0, c0,o,output = model(x,y[:,0,:2],h0,c0,ind = i,o = o,output=output)
      loss = torch.mean((out-y[:,1+(50*i):1+(50*(i+1)),:2])**2)
      writer.add_scalar("Loss/train", loss, epoch)
      
      loss.backward()
      optimizer.step()
    #print(y[:,0,:2])
    print('epoch',epoch,'seql',seql)
    print(loss)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/container.py:119: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


epoch 0 seql 11
tensor(1.9774, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 11
tensor(1.4379, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 11
tensor(1.5116, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 11
tensor(1.6626, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 11
tensor(1.4973, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 11
tensor(1.6600, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 11
tensor(1.4660, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 11
tensor(1.5318, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 11
tensor(1.7014, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 11
tensor(1.4997, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 11
tensor(1.4001, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 11
tensor(1.4938, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 11
tensor(1.4366, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 11
tensor(1.4982, device='cuda:0', gra

KeyboardInterrupt: ignored

In [ ]:
#optimizer = torch.optim.Adam(model.parameters(),lr = 0.00001, betas = (.9,.999))#ideal
for g in optimizer.param_groups:
    g['lr'] = 0.00001



In [ ]:

model = model.train()
#for seql in range(15,20,4):
seql = 11
for epoch in range (150):
  loss = None
  for x,y in loader_train:
    h0 = None
    c0 = None
    for i in range(seql):
      x = x.cuda()
      y = y.cuda()
      
      optimizer.zero_grad()
      
      if i == 0:
        out, h0, c0,o,output = model(x,y[:,0,:2],h0,c0,ind = i)
      else:
        out, h0, c0,o,output = model(x,y[:,0,:2],h0,c0,ind = i,o = o,output=output)
      loss = torch.mean((out-y[:,1+(50*i):1+(50*(i+1)),:2])**2)
      writer.add_scalar("Loss/train", loss, epoch)
      
      loss.backward()
      optimizer.step()
    #print(y[:,0,:2])
    print('epoch',epoch,'seql',seql)
    print(loss)

In [ ]:

#optimizer.zero_grad()


In [ ]:
"""model = model.train()
for epoch in range (20):
  loss = None
  for x,y in loader_train:
    h0 = None
    c0 = None
    for i in range(19):
      x = x.cuda()
      y = y.cuda()
      
      optimizer.zero_grad()
      #print(x.shape,y.shape)
      if i == 0:
        out, h0, c0, o = model(x,y[:,i*50:50*(i+1),:2],h0,c0,ind = i)
      else:
        out, h0, c0, o = model(x,y[:,i*50:50*(i+1),:2],h0,c0,ind = i,o=o)
        
      loss = torch.mean((out-y[:,1+(50*i):1+(50*(i+1)),:2])**2)
      writer.add_scalar("Loss/train", loss, epoch)

      loss.backward()
      optimizer.step()
    #print(y[:,0,:2])
    print('epoch',epoch)
    print(loss)"""

In [12]:
from torch.utils import data
from torch.utils.data import DataLoader, TensorDataset, RandomSampler

mini_batch = 100
test_dataset = DonutDataset(length = 100*10)
loader_test = data.DataLoader(
    test_dataset, 
    batch_size=mini_batch,
    sampler=RandomSampler(data_source=test_dataset),
    num_workers=2)


x0_y0_r_xrf_yrf torch.Size([1000, 1]) torch.Size([1000, 1]) torch.Size([1000, 1000]) torch.Size([1, 1000]) torch.Size([1, 1000])


In [13]:
model = model.eval()
with torch.no_grad():
  for epoch in range (1):
    loss = None
    for x,y in loader_test:
      h0 = None
      c0 = None
      seql = 11
      loss = torch.zeros(seql)
      for i in range(seql):
        optimizer.zero_grad()
        x = x.cuda()
        y = y.cuda()
        
        #print(x.shape,y.shape)
        if i == 0:
          out, h0, c0,o,output = model(x,y[:,0,:2],h0,c0,ind = i)
        else:
          out, h0, c0,o,output = model(x,y[:,0,:2],h0,c0,ind = i,o = o,output=output)
          
        loss[i] = torch.mean((out-y[:,1+(50*i):1+(50*(i+1)),:2])**2)
        print(loss[i])
        writer.add_scalar("Loss/val", loss[i],i, epoch)
        

      #print(y[:,0,:2])
      print('epoch',epoch)
      print('test loss', torch.mean(loss))

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/container.py:119: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
epoch 0
test loss tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
epoch 0
test loss tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
epoch 0
test loss tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
epoch 0
test loss tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
epoch 0
test loss tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
tensor(nan)
epoch 0
test loss tensor(nan)
tensor(nan)
tensor(nan)
tens

In [ ]:
writer.flush()
writer.close()


In [ ]:
#%load_ext tensorboard

In [ ]:
#!kill 3285
#%tensorboard --logdir runs

In [ ]:
pred_dataset = DonutDataset(length = 100)

In [ ]:
!nvidia-smi

In [ ]:
with torch.no_grad():
  model = model.eval()
  DonutDataset.displayCanvas('lstmpredictions.png',pred_dataset, model = model)

In [ ]:
with torch.no_grad():
  model = model.eval()
  DonutDataset.displayCanvas('trainlstmpredictions.png',dataset, model = model)

In [ ]:
for name, param in model.named_parameters():
  print(name)